In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [10]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [11]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[ 1, 73, 61, 54, 73,  1, 59, 62],
        [67, 73, 62, 58, 57,  1, 61, 62],
        [ 1, 58, 72, 56, 54, 69, 58,  0],
        [69, 54, 67, 62, 56,  1, 54, 67]], device='cuda:0')
targets:
tensor([[73, 61, 54, 73,  1, 59, 62, 72],
        [73, 62, 58, 57,  1, 61, 62, 66],
        [58, 72, 56, 54, 69, 58,  0, 74],
        [54, 67, 62, 56,  1, 54, 67, 57]], device='cuda:0')


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



N53D.6B﻿Rdz:XAu&Ionhi]k:i
[Np&u*﻿UY*
vo.fHVv_﻿PSv2h9uU"KhH?lTcdvpt?d_*6UR3W?LC):Frdh'Uk6 BkxE0,Se!t"-F9&
m[oi4JYee
u**bjVx:8Qu?mo!s_Tqa-aq'.RP?VAh3f&7ZBvuUO QYsv0a7Jd;9sp
m,e[iVUlEC)RN-g.RNgMj)KKocueYMn'CP4uleHgEFT;kzVSwb5P]AJ:;gVp][2,!HF!Bos'5?&!fE0sd!UO()yVvTy7rGh7gGL546﻿s?0a9I)y﻿]Fr?BZAzysvX.B ?EtvH9uUq5Wl!T0IU*M:Qu&﻿NU?.SYticJv)tAR;)io
jmAmvUr]"ULl Ww"L*E;L;9E!2k'0eAQ_5aAZ
Q][qWVRK﻿&l QQ]9G7J"uRticUL.x(EvTq&p EYBE,EN_R*(rXy2﻿P
]:;("(kJ]2F?jrbiWz&﻿ytv0leNLE0I*.pus[
Pd;L-,9w,!w7qB9qCw0_ueNB;l"


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.760, val loss: 4.747
step: 250, train loss: 4.719, val loss: 4.678
step: 500, train loss: 4.627, val loss: 4.629
step: 750, train loss: 4.563, val loss: 4.564
step: 1000, train loss: 4.515, val loss: 4.491
step: 1250, train loss: 4.459, val loss: 4.446
step: 1500, train loss: 4.395, val loss: 4.383
step: 1750, train loss: 4.331, val loss: 4.345
step: 2000, train loss: 4.285, val loss: 4.263
step: 2250, train loss: 4.230, val loss: 4.221
step: 2500, train loss: 4.159, val loss: 4.172
step: 2750, train loss: 4.132, val loss: 4.117
step: 3000, train loss: 4.088, val loss: 4.053
step: 3250, train loss: 4.038, val loss: 4.026
step: 3500, train loss: 3.994, val loss: 3.978
step: 3750, train loss: 3.943, val loss: 3.925
step: 4000, train loss: 3.882, val loss: 3.883
step: 4250, train loss: 3.851, val loss: 3.850
step: 4500, train loss: 3.807, val loss: 3.798
step: 4750, train loss: 3.761, val loss: 3.769
step: 5000, train loss: 3.719, val loss: 3.707
step: 5250, train l

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


XU.pE1jPWBUC:z-()O
Xc*gP]noeJ;'90FuL'7_fKZ*E1bW0y]!f5]*YY&CWMa-GJP8g e NLB
MixPBSBldQJPik2KZomADa3H2CV!5oa-,80gvQ,()cwoq-d)hpj?f18QSS;&t!KnBlXcwPreEhIel"_(DvYa3DpjguaTPBhpNx[C﻿[&:9ogfo70ygrdVE1-cQwoZaEwSj?6(﻿pbTfEmBS 4-*g
oc,7"zHbnJy:kty]f0rwa3jog];']1J]Vie﻿﻿3NhdT3jDa[0]1Q'V*c
sR"YwSPN!Fuv;;WVT.ylXEs wS,8_kW]MyWBuLaWJ!5fzRikjguL9jZNxu!Im5fj:5]I?e m5vdQ_faH;Laqn:0Im2qAizg!0Ijf3j6G:Vkik!*pj?QZojE1Ma1bHT'rYi]ojK'L:Ya3MO;'c[Wr,P1nVnoMP0wldung)7] 0djT,eoUdK!5jUR-[Zn2﻿35U"Pr&alG,W0wbmQsF73A*8[OuJSjUI5
